<a href="https://colab.research.google.com/github/jeelkukadiya/CodSoft-internship_ML/blob/main/Task_1_Movie_Genre_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:

# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES
# TO THE CORRECT LOCATION (/kaggle/input) IN YOUR NOTEBOOK,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'genre-classification-dataset-imdb:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F1417162%2F2347441%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240601%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240601T105535Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D0ae6d400b1cae0b2cd379226ebef339ebf841678a3aa35550c4b24d860dd24bb8e5977eabf77512a555d028fae4420d6e2d393fc8375243dc30eb30c4d512af179979a7d079180f6faff6723e6c674256403248a50bc5b0aae9c0924a2111ccc8518c73191fa2a2275f014860bf10b84669477d0c6ddb39b4961bc15139cd78541d04d9649d0ffa4b53f996aaa0db3108ec2b8da056d7f653e0f25c2dda642a6a1a9d261fd6d7a9763bedd91fbe4bb4dce2fd17df4c7a2400a9fab435e2f9521a1c2ddf8a29c0eb0219205472b6876144ea97a562acb09b6609c0777537912f473ae2cac8ae7710824b71a97f541b2d4b63252f2f345aada6a6dc610ae30adfa'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')


[==================================================] 43729690 bytes downloaded
Downloaded and uncompressed: genre-classification-dataset-imdb
Data source import complete.


In [2]:
import pandas as pd
import plotly.express as px
from sklearn.decomposition import PCA
from sklearn.feature_extraction.text import TfidfVectorizer

Set `nrows=6000` as ran out of memory when computing tokens with Tfidf on entire dataset.

In [3]:
df = pd.read_csv('/kaggle/input/genre-classification-dataset-imdb/Genre Classification Dataset/train_data.txt', sep=' ::: ', engine='python', names=['Title', 'Genre', 'Description'], nrows=6000)

df.head()

,Title,Genre,Description
1,Oscar et la dame rose (2009),drama,Listening in to a conversation between his doc...
2,Cupid (1997),thriller,A brother and sister with a past incestuous re...
3,"Young, Wild and Wonderful (1980)",adult,As the bus empties the students for their fiel...
4,The Secret Sin (1915),drama,To help their unemployed father make ends meet...
5,The Unrecovered (2007),drama,The film's title refers not only to the un-rec...


In [4]:
df['Genre'].value_counts()

Genre
drama          1481
documentary    1459
comedy          830
short           559
horror          235
thriller        211
action          146
western         116
reality-tv      100
family           82
adventure        82
sci-fi           79
music            75
adult            73
romance          58
animation        54
sport            51
crime            48
talk-show        41
mystery          37
biography        36
fantasy          35
musical          29
history          27
game-show        25
news             18
war              13
Name: count, dtype: int64

Choose to keep only these Genres as they provide a nice visualisation using PCA.

In [5]:
df = df[
    (df['Genre'] == 'drama') | (df['Genre'] == 'music') | (df['Genre'] == 'documentary') | (df['Genre'] == 'western')]

In [6]:
vec = TfidfVectorizer(stop_words='english')

In [7]:
matrix = vec.fit_transform(df['Description'])

In [8]:
X = matrix.toarray()

Unique words in dataset.

In [9]:
len(vec.get_feature_names_out())

27288

In [10]:
vec.get_feature_names_out()

array(['00', '000', '000km', ..., 'żestán', 'żo', 'żte'], dtype=object)

# 2 Component PCA

In [11]:
pca = PCA(n_components=2)
X_pca = pca.fit_transform(X)

In [12]:
fig = px.scatter(x=X_pca[:, 0], y=X_pca[:, 1], color=df['Genre'], hover_name=df['Title'], template='plotly_dark')
fig.update_layout(
    title="2 Component PCA visualization of Movie Genres",
    xaxis_title="1st Principal Component",
    yaxis_title="2nd Principal Component",
)
fig.show()

# 3 Component PCA

In [13]:
pca = PCA(n_components=3)
X_pca = pca.fit_transform(X)

In [14]:
fig = px.scatter_3d(x=X_pca[:, 0], y=X_pca[:, 1], z=X_pca[:, 2], color=df['Genre'], opacity=0.8,
                    title="3 Component PCA visualization of Movie Genres", hover_name=df['Title'], template='plotly_dark')
fig.show()